In [1]:
import pandas as pd

df = pd.read_csv("/content/Creditcard_data.csv")
print(df.shape)
df.head()


(772, 31)


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,1
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [2]:
X = df.drop("Class", axis=1)
y = df["Class"]

print(y.value_counts())

Class
0    763
1      9
Name: count, dtype: int64


In [3]:
from sklearn.model_selection import train_test_split

samples = []
for i in range(5):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=i, stratify=y
    )
    samples.append((X_train, X_test, y_train, y_test))

print("5 Samples Created Successfully")


5 Samples Created Successfully


In [4]:
!pip install imbalanced-learn

In [5]:
from imblearn.under_sampling import RandomUnderSampler, NearMiss, TomekLinks
from imblearn.over_sampling import RandomOverSampler, SMOTE

samplers = {
    "Sampling1": RandomUnderSampler(random_state=42),   # Random UnderSampling
    "Sampling2": RandomOverSampler(random_state=42),    # Random OverSampling
    "Sampling3": SMOTE(random_state=42),                # SMOTE
    "Sampling4": NearMiss(),                            # NearMiss
    "Sampling5": TomekLinks()                           # Tomek Links
}


In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

models = {
    "M1": LogisticRegression(max_iter=2000),
    "M2": DecisionTreeClassifier(random_state=42),
    "M3": RandomForestClassifier(n_estimators=200, random_state=42),
    "M4": SVC(kernel="rbf"),
    "M5": GaussianNB()
}

In [7]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import numpy as np

results = pd.DataFrame(index=models.keys(), columns=samplers.keys())

for m_name, model in models.items():
    for s_name, sampler in samplers.items():

        acc_list = []

        for X_train, X_test, y_train, y_test in samples:

            X_res, y_res = sampler.fit_resample(X_train, y_train)

            if m_name in ["M1", "M4"]:   # scaling for LR and SVM
                scaler = StandardScaler()
                X_res = scaler.fit_transform(X_res)
                X_test_scaled = scaler.transform(X_test)

                model.fit(X_res, y_res)
                y_pred = model.predict(X_test_scaled)

            else:
                model.fit(X_res, y_res)
                y_pred = model.predict(X_test)

            acc_list.append(accuracy_score(y_test, y_pred))

        results.loc[m_name, s_name] = round(np.mean(acc_list) * 100, 2)

results

,Sampling1,Sampling2,Sampling3,Sampling4,Sampling5
M1,37.76,90.0,90.6,59.91,98.62
M2,67.33,97.67,97.41,41.12,97.5
M3,48.88,98.71,98.53,44.74,98.71
M4,38.19,95.95,95.86,45.17,98.71
M5,68.28,88.97,91.21,28.71,93.79


In [8]:
best_sampling = results.astype(float).idxmax(axis=1)
best_accuracy = results.astype(float).max(axis=1)

final_best = pd.DataFrame({
    "Best Sampling Technique": best_sampling,
    "Highest Accuracy": best_accuracy
})

final_best

,Best Sampling Technique,Highest Accuracy
M1,Sampling5,98.62
M2,Sampling2,97.67
M3,Sampling2,98.71
M4,Sampling5,98.71
M5,Sampling5,93.79


In [10]:
model_names = {
    "M1": "Logistic Regression",
    "M2": "Decision Tree",
    "M3": "Random Forest",
    "M4": "Support Vector Machine (SVM)",
    "M5": "Naive Bayes"
}

best_sampling = results.astype(float).idxmax(axis=1)
best_accuracy = results.astype(float).max(axis=1)

print("Best Sampling Technique for Each Model:\n")

for m in results.index:
    print(f"{m} ({model_names[m]}) -> Best: {best_sampling[m]} with Accuracy = {best_accuracy[m]:.2f}%")


Best Sampling Technique for Each Model:

M1 (Logistic Regression) -> Best: Sampling5 with Accuracy = 98.62%
M2 (Decision Tree) -> Best: Sampling2 with Accuracy = 97.67%
M3 (Random Forest) -> Best: Sampling2 with Accuracy = 98.71%
M4 (Support Vector Machine (SVM)) -> Best: Sampling5 with Accuracy = 98.71%
M5 (Naive Bayes) -> Best: Sampling5 with Accuracy = 93.79%
